In [1]:
import sys
from os import path
import os
from scipy.spatial.distance import cdist
import numpy as np
import datetime
import pandas as pd
import time
import math
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('whitegrid')
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy.stats import gaussian_kde
import matplotlib.lines as mlines
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from matplotlib.offsetbox import AnchoredText
from datetime import datetime as dt
import utide

In [2]:
def datenumaz(d):
    return 366 + d.toordinal() + (d - dt.fromordinal(d.toordinal())).total_seconds()/(24*60*60)

In [3]:
basefol = '/Users/amrozeidan/Documents/hiwi/easygshpy/base'

In [8]:
df_simul_matlab_a = pd.read_csv('/Users/amrozeidan/Downloads/EAZYgsh tools/testing_for_nico/base/coef_simulated/A_simul_all_stations.dat')
df_simul_matlab_a.set_index('tide' , inplace = True)
df_simul_matlab_a.head()

,Emshoern,Eemshaven,Dukegat,Delfzijl,BorkumS
tide,,,,,
MM,289.555357,292.882322,310.621050,334.931412,244.696079
NU2,191.368253,194.597297,199.142857,206.131317,168.011043
N2,181.375247,184.479048,188.825634,195.270159,159.121604
MF,42.312405,42.784532,45.332860,48.703205,35.877572
M2,23.012729,23.367411,23.926685,25.040884,20.260128


In [11]:
a_mat = df_simul_matlab_a.loc[: , ['BorkumS']]
a_mat.columns = ['a_matlab']

In [12]:
df_simul_matlab_g = pd.read_csv('/Users/amrozeidan/Downloads/EAZYgsh tools/testing_for_nico/base/coef_simulated/g_simul_all_stations.dat')
df_simul_matlab_g.set_index('tide' , inplace = True)
df_simul_matlab_g.head()

,Emshoern,Eemshaven,Dukegat,Delfzijl,BorkumS
tide,,,,,
MM,96.804670,96.812702,96.795143,96.787278,96.815861
NU2,275.304019,276.175337,281.189305,286.058272,262.336655
N2,188.269355,189.147377,194.194966,199.136112,175.251248
MF,51.924798,51.950114,51.930936,52.009440,51.879608
M2,5.236390,6.040082,10.897496,15.668607,352.443757


In [13]:
g_mat = df_simul_matlab_g.loc[: , ['BorkumS']]
g_mat.columns = ['g_matlab']

In [29]:
df_a_py = pd.read_csv(basefol+'/dfa_py.dat')
df_a_py.head()

,Unnamed: 0,BorkumS,BorkumS.1,Emshoern,Emshoern.1,Eemshaven,Eemshaven.1,Dukegat,Dukegat.1,Delfzijl,Delfzijl.1
0,NaN,meas,simul,meas,simul,meas,simul,meas,simul,meas,simul
1,2MS6,0.054817874647307335,0.05572999164538584,0.04971855196024329,0.04803944891300183,0.052349795680099016,0.0494299865788257,0.05701029183532341,0.04813724304861886,0.10425981670352806,0.08647339185443555
2,2SM2,0.2755892081563844,0.2594293903296893,0.2732035610525629,0.2960051242393389,0.2578044940643594,0.29945307679923416,0.27875334864628304,0.3077639915876301,0.13953226410260297,0.36810588538277794
3,K1,0.21186915466496725,0.1593117191635717,0.20030178559809397,0.13745955871907264,0.20149004531682937,0.1361222976986955,0.19236802755159513,0.12457679655475436,0.19013367807588868,0.09786482747700842
4,M10,0.0019068788111273247,0.0026498266319605586,0.0031353282285444934,0.00337060443635096,0.0019607230828983567,0.003807882111307833,0.003944126343862721,0.004710968498418963,0.006364463146764243,0.0050980799452038416


In [28]:
a_py = df_a_py.iloc[: , [0 ,2 ]]
a_py.columns = ['tide' , 'a_py']
a_py.set_index('tide' , inplace=True)
a_py

,a_py
tide,
NaN,simul
2MS6,0.05572999164538584
2SM2,0.2594293903296893
K1,0.1593117191635717
M10,0.0026498266319605586
M12,0.0013911044261073564
M2,13.47395137519856
M4,0.557781271242576
M6,0.024167992874897088


In [30]:
df_g_py = pd.read_csv(basefol+'/dfg_py.dat')
df_g_py.head()

,Unnamed: 0,BorkumS,BorkumS.1,Emshoern,Emshoern.1,Eemshaven,Eemshaven.1,Dukegat,Dukegat.1,Delfzijl,Delfzijl.1
0,NaN,meas,simul,meas,simul,meas,simul,meas,simul,meas,simul
1,2MS6,108.46742506874962,119.55407093206067,158.8580456517444,147.48086675089075,158.40592372608054,150.9571697727624,174.76814694427782,167.30487773640115,202.68185968780463,225.50335100838623
2,2SM2,149.88657937045213,122.71585177359488,147.7482307525153,131.41005837845645,152.15429088143037,131.80305061306746,145.16721339889745,134.85551355710325,153.82070923863722,126.73974102802931
3,K1,210.88027985537707,250.18568323597145,217.93105946221212,258.21024448229423,214.2881846337084,258.9575066001038,214.6552961227961,260.03097404953047,222.04528312563778,288.42818836407093
4,M10,216.67331204954985,276.9025169231929,332.47412429598666,5.509945563220957,338.9480530326247,10.48260675696907,348.7078320225863,27.68841494823077,20.94389413411501,88.99402472845956


In [31]:
g_py = df_g_py.iloc[: , [0 ,2 ]]
g_py.columns = ['tide' , 'g_py']
g_py.set_index('tide' , inplace=True)
g_py

,g_py
tide,
NaN,simul
2MS6,119.55407093206067
2SM2,122.71585177359488
K1,250.18568323597145
M10,276.9025169231929
M12,190.50127948232404
M2,74.22041369495278
M4,158.33772567154293
M6,226.0302014581919


In [32]:
dfmat = a_mat.join(g_mat , how='inner')
dfpy = a_py.join(g_py , how = 'inner')
dfall = dfmat.join(dfpy , how = 'inner')
dfall = dfall.sort_index(axis=1)

In [33]:
dfall

,a_matlab,a_py,g_matlab,g_py
tide,,,,
MM,244.696079,242.57922359750577,96.815861,197.54386660621958
NU2,168.011043,107.40869668775686,262.336655,163.57764983847287
N2,159.121604,102.37118672762836,175.251248,156.23540214802208
MF,35.877572,35.5693508159442,51.879608,334.8599072568241
M2,20.260128,13.47395137519856,352.443757,74.22041369495278
MU2,9.055363,5.571103929719692,85.587771,243.5970130701205
MK4,4.215817,4.453263082889729,280.933255,48.57623566182802
MS4,3.370687,3.4903197721471453,238.599329,10.271627917338453
S2,2.287778,1.6184291398019475,274.415073,289.65512412319646
